# REviews SEntiment analysis

## Import all the required librarires

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import matplotlib.pyplot as plt
# api key of rapid api image generatro =bcf1e9134fmsh7a3274be12a7e8ap1220afjsnc3125374e8b1

## combining all the other csv file together

In [18]:


csv_files = ['fantasy.csv', 'action.csv', 'adult.csv', 'adventure.csv', 'animation.csv', 
             'biography.csv', 'crime.csv', 'film-noir.csv', 'game-show.csv', 'history.csv', 
             'horror.csv', 'music.csv', 'musical.csv', 'mystery.csv', 
             'reality-tv.csv', 'sci-fi.csv', 'sport.csv', 'thriller.csv', 'war.csv', 'western.csv']

dfs = []


for file in csv_files:
    df = pd.read_csv(file, dtype={'votes': str, 'gross_income': str})[['id', 'name']]
    dfs.append(df)


df_combined = pd.concat(dfs, ignore_index=True)

df_combined = df_combined[['id', 'name']]

df_combined['name'].fillna('', inplace=True)

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_combined['name'])

# # Print the combined DataFrame
# print(df_combined)


C:\Users\Dell\AppData\Local\Temp\ipykernel_9328\315964168.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_combined['name'].fillna('', inplace=True)


## it is used to find the most closest movie id with respect to its name

In [19]:

movies_dict = dict(zip(df_combined['name'].str.lower(), df_combined['id']))

def get_closest_movie_id(movie_name):
   
    movie_name_lower = movie_name.lower()

   
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df_combined['name'])

 
    input_vector = vectorizer.transform([movie_name_lower])
    similarity_scores = cosine_similarity(input_vector, tfidf_matrix)

    most_similar_index = similarity_scores.argmax()


    closest_movie_id = df_combined.iloc[most_similar_index]['id']
    closest_movie_name = df_combined.iloc[most_similar_index]['name']

    return closest_movie_id, closest_movie_name

## to scrap reviews from imdb from a particular id

In [20]:

def scrape_reviews(movie_name):
    url = f"https://www.imdb.com/title/{movie_name}/reviews"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    reviews = []
    for review in soup.find_all('div', class_='text show-more__control'):
        reviews.append(review.text.strip())
    return reviews

def analyze_sentiment(reviews):
    positive_count = 0
    negative_count = 0
    neutral_count = 0
    for review in reviews:
        analysis = TextBlob(review)
        if analysis.sentiment.polarity > 0:
            positive_count += 1
        elif analysis.sentiment.polarity < 0:
            negative_count += 1
        else:
            neutral_count += 1
    total_reviews = len(reviews)
    return (positive_count, negative_count, neutral_count, total_reviews)


def is_worth_watching(positive_count, negative_count, neutral_count, total_reviews):
    if total_reviews == 0:
        return "No reviews available."
    elif positive_count / total_reviews >= 0.6:
        return "Worth watching!"
    else:
        return "Not worth watching!"





In [21]:

def Other_main(movie_name):
    
    reviews = scrape_reviews(movie_name)
    positive_count, negative_count, neutral_count, total_reviews = analyze_sentiment(reviews)
    verdict = is_worth_watching(positive_count, negative_count, neutral_count, total_reviews)
    print("Sentiment Analysis Result:")
    print(f"Positive Reviews: {positive_count}")
    print(f"Negative Reviews: {negative_count}")
    print(f"Neutral Reviews: {neutral_count}")
    print(f"Total Reviews: {total_reviews}")
    print(verdict)
    # return reviews
    # print(reviews)

In [22]:

def sentiment_on_movie(movie_name):
    closest_movie_id, closest_movie_name = get_closest_movie_id(movie_name)
    Other_main(closest_movie_id)
    print(f"The closest movie to '{movie_name}' is '{closest_movie_name}' with ID: {closest_movie_id}")

In [27]:
sentiment_on_movie("super man man of steel")

Sentiment Analysis Result:
Positive Reviews: 24
Negative Reviews: 1
Neutral Reviews: 0
Total Reviews: 25
Worth watching!
The closest movie to 'super man man of steel' is 'Man of Steel' with ID: tt0770828


In [24]:
import pickle



csv_files = ['musical.csv', 'adult.csv', 'adventure.csv', 'animation.csv', 
             'biography.csv', 'crime.csv', 'film-noir.csv', 'game-show.csv', 
             'history.csv', 'horror.csv', 'music.csv', 'western.csv', 'war.csv','fantasy.csv', 
             'thriller.csv', 'sport.csv', 'sci-fi.csv', 'reality-tv.csv', 'mystery.csv']

dfs = []


for file in csv_files:
    df = pd.read_csv(file)
    df = df[['name']]
    df.loc[:, 'name'].fillna('', inplace=True)
    pickle.dump(df.to_dict(), open(f'movies_title_{file[:-4]}.pkl', 'wb'))



C:\Users\Dell\AppData\Local\Temp\ipykernel_9328\1925765848.py:15: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\Dell\AppData\Local\Temp\ipykernel_9328\1925765848.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.loc[:, 'name'].fillna('', inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_9328\1925765848.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplac

In [25]:

pickle.dump(df_combined.to_dict(),open('movies_title.pkl','wb'))
df_combined

,id,name
0,tt4574334,Stranger Things
1,tt1312171,The Umbrella Academy
2,tt9419884,Doctor Strange in the Multiverse of Madness
3,tt10857164,Ms. Marvel
4,tt10648342,Thor: Love and Thunder
...,...,...
3561856,tt9660040,Wildcards
3561857,tt9660044,Wildcards
3561858,tt9660046,Wildcards
3561859,tt9660050,Wildcards


In [26]:
movies_list = pickle.load(open('movies_title.pkl','rb'))
movies = pd.DataFrame(movies_list)
movies

,id,name
0,tt4574334,Stranger Things
1,tt1312171,The Umbrella Academy
2,tt9419884,Doctor Strange in the Multiverse of Madness
3,tt10857164,Ms. Marvel
4,tt10648342,Thor: Love and Thunder
...,...,...
3561856,tt9660040,Wildcards
3561857,tt9660044,Wildcards
3561858,tt9660046,Wildcards
3561859,tt9660050,Wildcards
